# Visión por Computador - Práctica 1
## Filtrado y muestreo

### A) Implementar una función de convolución que debe ser capaz de calcular la convolución 2D de una imagen con una máscara. Ahora supondremos que la máscara es extraída del muestreo con una Gaussiana 2D simétrica. Para ello implementaremos las siguientes funciones auxiliares:

#### 1. Cálculo del vector máscara: Sea $f(x) = e^{\left(-0.5\frac{x^2}{\sigma^2}\right)}$ una funcion donde $\sigma$ representa un parámetro en unidades píxel. Implementar una función que tomando $\sigma$ como parámetro de entrada devuelva una máscara de convolución representativa de dicha función. Justificar los pasos dados.

#### 2. Implementar una función que calcule la convolución de un vector señal 1D con un vector-máscara 1D de longitud inferior al de la señal usando dos posibles tipos de condiciones de entorno (uniforme a ceros y reflejada). La salida será un vector de igual longitud que el vector señal de entrada. Pasos:

   ##### a) Implementar una función que calcule la convolución 1D entre dos vectores, dando de salida solo los valores donde ha sido posible el cálculo.
    
   ##### b) Definir un vector auxilar de longitud mayor definida a partir de las dimensiones de los dos vectores de entrada, p.e. la señal tiene longitud $N$ y el otro $2k + 1$ entonces el vector auxiliar será $N+2k$. Copiar en su centro el vector señal y rellenar los extremos usando el criterio de borde elegido. 
    
   ##### c) Ejecutar la función del paso a sobre el vector creado y la máscara. En el caso de que el vector de entrada sea de color, habrán de extraerse cada uno de los tres vectores correspondientes a cada uno de ellos y volver a montar el vector de salida. Usar las funciones *split()* y *merge()* de openCV.
    
#### 3. Implementar una función que tomando como entrada una imagen y el valor de $\sigma$ calcule la convolución de dicha imagen con uma máscara Gaussiana 2D. Usar las funciones implementadas en el punto anterior. Recordar que la Gaussiana es descomponible en convoluciones 1D por filas y columnas.

- **Apartado 1**: Para resolver el apartado 1, se ha creado la función ***get_mask_vector*** que recibe como parámetro de entrada el valor de $\sigma$. Una vez dentro de la función, muestrearemos $f(x)$ entre los valores $-3\sigma$ y $3\sigma$.

    Para ello, en el intervalo $\left[\lfloor-3\sigma\rfloor, \lceil3\sigma\rceil\right]$, muestrearemos los $n$ valores enteros que haya en este intervalo, obteniendo un vector de enteros. Sobre este vector de enteros, aplicaremos la función $f(x)$ a este vector obteniendo así la máscara, pero sin normalizar.

    Una vez obtenida, los valores que se encuentren por debajo de $|3\sigma|$, los llevaremos a 0, ya que consideraremos que son poco relevantes y que influyen muy poco en el resultado. Tras esto, normalizaremos a 1 la máscara y se devolverá.

In [1]:
# Devuelve el vector gaussiano de la máscara
def get_mask_vector(sigma):
    # Obtenemos el valor "límite" que puede tener un
    # píxel para que sea significativo
    limit_3sigma = fx(3 * sigma, sigma)
    # Obtenemos un array de valores discretos
    # para realizar la gaussiana
    aux = np.arange(math.floor(-3 * sigma), math.ceil(3 * sigma )+ 1)
    # Rellenamos la máscara aplicando la exponencial
    mask = np.array([fx(i, sigma) for i in aux])
    # Despreciamos los valores menores a 3 sigma
    mask[mask < limit_3sigma] = 0
    # Normalizamos la máscara a 1
    mask = np.divide(mask, np.sum(mask))
    # Devolvemos la máscara
    return mask

- **Apartado 2 - a**: para resolver el apartado dos, primero tenemos que fijarnos en cómo se realiza una convolución. $$G[i,j] = \sum_{u=-k}^k\sum_{v=-k}^kH[u,v]F[i-u,j-v]$$

    Al ser una función Gaussiana, tendremos una máscara con valores simétricos, por lo que podemos simplificar la expresión haciendo que la convolución se resuma en multiplicar la máscara por el vector 1D, y obtener la suma del vector resultante, siendo esta el valor de $G[i,j]$.

In [1]:
def convolution(mask, img_array):
    return np.sum(img_array * mask)

En esta función, se recibe como parámetro la máscara gaussiana, y un array 1D perteneciente a la imagen, y devuelve el valor que se le asignará al píxel $G_{ij}$.

- **Apartado 2 - b**: para definir este vector auxiliar, se han generado varias funciones para esta tarea. Una de ellas generará la imagen auxiliar de tamaño $N+2k$ y copiará en su centro la imagen original, y el resto se encargarán de rellenar los bordes de la imagen auxiliar de una forma u otra. En OpenCV existen distintas formas de rellenar los bordes:
    * **Replicado**: replica el último píxel que hay en el borde de la imagen $n$ veces.
    * **Reflejado**: coge los $n$ últimos píxeles del borde y los añade en orden inverso.
    * **Reflejado 101**: similar al anterior, pero añade los píxeles en el lado contrario del que se toman.
    * **Wrap o envoltura**: toma los $n$ píxeles cercanos al borde y los añade en el mismo orden al lado inverso de la imagen.
    * **Constante**: añade a los bordes píxeles con un valor constante $k$.
 
 Para empezer, la función ***extend_image_n_pixels*** recibe como parámetros la imagen original, el número de píxeles que se desea extender, el tipo de borde que se desea (que se codifica con un entero), y una constante $k$ para rellenar los píxeles con esa constante en caso de que se elija esta opción, que por defecto es 0. Esta función, distingue entre imágenes en color e imágenes en escala de grises.
     * **Imágenes en escala de grises**: llama a la función ***extend*** y genera la imagen extendida según los parámetros recibidos.
     * **Imágenes en color**: utilizando la función ***split***, obtenemos el canal azul, verde y rojo de la imagen original, y con cada uno de ellos se llama a la función ***extend***. Una vez extendidos los tres canales, se unen usando la función ***merge*** de OpenCV.
     
 La función ***extend*** recibe la imagen (o canal) que queremos ampliar, el número de píxeles, el borde y la constante que se usa en caso de se use la extensión con una constante. Genera una matriz auxiliar con de ceros e inserta la imagen en el centro usando la función ***insert_img_into_other***. Tras esto, extiende la imagen según el criterio elegido.

In [3]:
def insert_img_into_other(img_src, pixel_left_top_row, 
                          pixel_left_top_col, 
                          img_dest,substitute=False):
    alt, anch = img_src.shape[:2]
    if not substitute:
        img_dest[pixel_left_top_row:alt+pixel_left_top_row,
                 pixel_left_top_col:anch+pixel_left_top_col] += img_src
    else:
        img_dest[pixel_left_top_row:alt + pixel_left_top_row,
                 pixel_left_top_col:anch + pixel_left_top_col] = img_src
        
# Border replicate => coge el último píxel y lo replica
# n_pixels veces
def border_replicate(img, n_pixels, alt, anch):
    img[0:n_pixels, ] = img[n_pixels,]
    img[n_pixels + alt:(alt + 2 * n_pixels), ] = img[-n_pixels-1,]
    for i in range(n_pixels):
        img[:, i] = img[:, n_pixels]
    for i in range(n_pixels + anch, img.shape[1]):
        img[:, i] = img[:, -n_pixels-1]

# Border reflect => refleja n_pixels del borde y los añade
# al borde por ambos lados
def border_reflect(img, n_pixels, alt, anch):
    # Borde superior
    img[0:n_pixels, ] = np.matrix(img[n_pixels:n_pixels * 2, ])[::-1]
    # Borde inferior
    img[n_pixels + alt:(alt + 2 * n_pixels), ] = \
        np.matrix(img[(-2 * n_pixels):(-n_pixels), ])[::-1]
    # Para invertir los bordes de los laterales, no podemos usar el "atajo" de [::-1] para
    # invertir la matriz. Para ello, usaremos la función fliplr de numpy que nos devuelve
    # la matriz con las columnas en orden inverso
    # Borde izquierdo
    img[:, 0:n_pixels] = np.fliplr(img[:, n_pixels:2 * n_pixels])
    # Borde derecho
    img[:, n_pixels + anch:img.shape[1]] = \
        np.fliplr(img[:, (-2 * n_pixels):(-n_pixels)])

# Border reflect 101 => similar al anterior, pero, refleja
# el borde en el borde contrario de la imagen
def border_reflect_101(img, n_pixels, alt, anch):
    # Borde superior
    img[0:n_pixels, ] = np.matrix(img[(-2 * n_pixels):(-n_pixels), ])[::-1]
    # Borde inferior
    img[n_pixels + alt:(alt + 2 * n_pixels), ] = \
        np.matrix(img[n_pixels:n_pixels * 2, ])[::-1]
    # Borde izquierdo
    img[:, 0:n_pixels] = np.fliplr(img[:, (-2 * n_pixels):(-n_pixels)])
    # Borde derecho
    img[:, n_pixels + anch:img.shape[1]] = \
        np.fliplr(img[:, n_pixels:2 * n_pixels])

# Border wrap => toma los últimos n_pixels de un borde de la
# imagen, y los añade tal cual al otro borde de la imagen
def border_wrap(img, n_pixels, alt, anch):
    # Borde superior
    img[0:n_pixels, ] = np.matrix(img[(-2 * n_pixels):(-n_pixels), ])
    # Borde inferior
    img[n_pixels + alt:(alt + 2 * n_pixels), ] = \
        np.matrix(img[n_pixels:n_pixels * 2, ])
    # Borde izquierdo
    img[:, 0:n_pixels] = img[:, (-2 * n_pixels):(-n_pixels)]
    # Borde derecho
    img[:, n_pixels + anch:img.shape[1]] = \
        img[:, n_pixels:2 * n_pixels]


# Border constant => a partir de una constante k, añade a los bordes
    # píxeles con valor k
def border_constant(img, n_pixels, alt, anch, k):
    # Borde superior
    img[0:n_pixels, ] = np.ones((n_pixels, img.shape[1]), dtype=np.uint8) * k
    # Borde inferior
    img[n_pixels + alt:(alt + 2 * n_pixels), ] = \
        np.ones((n_pixels, img.shape[1]), dtype=np.uint8) * k
    # Borde izquierdo
    img[:, 0:n_pixels] = np.ones((img.shape[0], n_pixels), dtype=np.uint8) * k
    # Borde derecho
    img[:, n_pixels + anch:img.shape[1]] = \
        np.ones((img.shape[0], n_pixels), dtype=np.uint8) * k


def extend(img_src, n_pixels, border_type, k):
    # Tomamos la altura y el ancho de la imagen original
    alt, anch = img_src.shape[:2]
    # Generamos una nueva matriz expandida a partir del
    # alto y ancho de la anterior
    new_extended_img = np.zeros((alt + 2 * n_pixels, 
                                 anch + 2 * n_pixels), dtype=np.uint8)

    insert_img_into_other(img_src=img_src, pixel_left_top_row=n_pixels,
                          pixel_left_top_col=n_pixels, img_dest=new_extended_img)

    if border_type == 0:    # Border Replicate
        border_replicate(new_extended_img, n_pixels, alt, anch)
    elif border_type == 1:  # Border Reflect
        border_reflect(new_extended_img, n_pixels, alt, anch)
    elif border_type == 2:  # Border Reflect 101
        border_reflect_101(new_extended_img, n_pixels, alt, anch)
    elif border_type == 3:  # Border wrap
        border_wrap(new_extended_img, n_pixels, alt, anch)
    elif border_type == 4:  # Border Constant
        border_constant(new_extended_img, n_pixels, alt, anch, k)

    return new_extended_img


def extend_image_n_pixels(img_src, n_pixels, border_type, k=0):

    if len(img_src.shape) != 3: # Imagen en escala de grises
        return extend(img_src, n_pixels, border_type, k)
    
    else:   # Imagen en color
        b_channel, g_channel, r_channel = cv2.split(img_src)

        b_channel_ext = extend(b_channel, n_pixels, border_type, k)
        g_channel_ext = extend(g_channel, n_pixels, border_type, k)
        r_channel_ext = extend(r_channel, n_pixels, border_type, k)

        return cv2.merge((b_channel_ext, g_channel_ext, r_channel_ext))


- **Apartado 2 - c**: la función definida para pasar la máscara a la imagen es ***convolution_grey_scale_img*** y ***convolution_color_img***, que reciben como parámetros la máscara, las imágenes extendidas, el número de píxeles que se ha extendido la imágen que entra como parámetro. Tras esto, se genera una copia de la imagen, se realiza la convolución en horizontal, se genera otra copia auxiliar, y se realiza la convolución en vertical. Si la imagen es en color o en escala de grises, tenemos las siguientes diferencias:
    * **Imágenes en color**: separa la imagen con la función ***split*** y se aplica la máscara a cada uno de los canales de color por separado, para luego volver a juntar los canales en una sola imagen con la función ***merge*** de OpenCV.
    * **Imágenes en escala de grises**: directamente aplica el filtro a la imagen, ya que al ser un único canal con la intensidad de cada píxel, puede realizarse sin tener que separar la imagen.

Al final, devolvemos la imagen resultante sin los bordes que se añadieron para poder hacer la convolución.

In [1]:
def convolution_grey_scale_img(mask, img, n_pixels):
    # Obtenemos el alto y ancho de la imagen
    alt_ext, anch_ext = img.shape[:2]

    cv = convolution
    copy = np.copy

    # Obtenemos una matriz auxiliar
    aux = copy(img)

    for i in range(n_pixels, alt_ext - n_pixels):
        for j in range(n_pixels, anch_ext - n_pixels):
            aux[i, j] = cv(mask, img[i, j - n_pixels:1 + j + n_pixels])

    aux_2 = copy(aux)

    for j in range(n_pixels, anch_ext - n_pixels):
        for i in range(n_pixels, alt_ext - n_pixels):
            aux_2[i, j] = cv(mask, aux[i - n_pixels:1 + i + n_pixels, j])

    return aux_2[n_pixels:-n_pixels, n_pixels:-n_pixels]


def convolution_color_img(mask, img, n_pixels):
    # Obtenemos el alto y ancho de la imagen
    alt_ext, anch_ext = img.shape[:2]

    # Separamos los colores de la imagen en tres canales
    b_channel, g_channel, r_channel = cv2.split(img)

    cv = convolution
    copy_img = np.copy
    zeros = np.zeros

    # Obtenemos una matriz auxiliar por cada uno de
    # los canales de color de la imagen
    aux_r = zeros((alt_ext, anch_ext), dtype=np.uint8)
    aux_g = zeros((alt_ext, anch_ext), dtype=np.uint8)
    aux_b = zeros((alt_ext, anch_ext), dtype=np.uint8)
    # Empezamos
    for i in range(n_pixels, alt_ext - n_pixels):
        for j in range(n_pixels, anch_ext - n_pixels):
            aux_b[i, j] = cv(mask, b_channel[i, j - n_pixels:1 + j + n_pixels])
            aux_g[i, j] = cv(mask, g_channel[i, j - n_pixels:1 + j + n_pixels])
            aux_r[i, j] = cv(mask, r_channel[i, j - n_pixels:1 + j + n_pixels])

    # Volvemos a tomar los valores de los bordes
    r_channel, g_channel, b_channel = copy_img(aux_r), copy_img(aux_g), copy_img(aux_b)
    # Y obtenemos nuevas matrices auxilares para terminar la convolucion
    aux2_r, aux2_g, aux2_b = copy_img(r_channel), copy_img(g_channel), copy_img(g_channel)

    for j in range(n_pixels, anch_ext - n_pixels):
        for i in range(n_pixels, alt_ext - n_pixels):
            aux2_b[i, j] = cv(mask, b_channel[i - n_pixels:1 + i + n_pixels, j])
            aux2_g[i, j] = cv(mask, g_channel[i - n_pixels:1 + i + n_pixels, j])
            aux2_r[i, j] = cv(mask, r_channel[i - n_pixels:1 + i + n_pixels, j])

    # Recomponemos la imagen
    result = cv2.merge((aux2_b, aux2_g, aux2_r))

    # Y la devolvemos
    return result[n_pixels:-n_pixels, n_pixels:-n_pixels]



- **Apartado 3**: con las funciones anteriormente definidas, podemos crear la función ***my_im_gauss_convolution*** encargada de realizar la convolución con la máscara 2D Gaussiana. Esta función recibe como parámetros la imagen original, la máscara gaussiana y el tipo de borde. Una vez dentro, calcula el número de píxeles que debe añadir a la imagen en cada borde, la extiende según el borde elegido y a continuación realiza la convolución. Si la imagen es en color, la matriz asociada a la imagen tendrá tres dimensiones, por lo que es fácil saber si una imagen es en color o en escala de grises, viendo si su ***shape*** o forma tiene una longitud igual a 3, lo que significa que es en color y se llamará a la función ***convolution_color_img*** para realizar la convolución. En caso contrario, se llamará a ***convolution_grey_scale_img***.

In [2]:
def my_im_gauss_convolution(im, mask_convolution,
                            border_type=0):
    # Extendemos los bordes de la imagen para hacer posible el uso de la
    # mascara sobre esta
    n_pixels = math.floor(mask_convolution.size / 2)

    extended_image = extend_image_n_pixels(im, n_pixels, border_type)
    # Si la imagen está en blanco y negro,
    if len(im.shape) != 3:
        return convolution_grey_scale_img(mask_convolution,
                                          extended_image, n_pixels)
    else:
        return convolution_color_img(mask_convolution,
                                     extended_image, n_pixels)